In [1]:
ls

data.csv  sample_data/


In [ ]:
!pip install torch_geometric

In [3]:
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_dense_adj, to_networkx
from torch_geometric.datasets import KarateClub
from torch.nn import Linear
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.optim as optim
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import torch.nn.functional as F
import math


In [4]:

nltk.download('punkt')
nltk.download('stopwords')

# Tách từ và loại bỏ các từ dừng
def tokenize_text(text):
    tokens = word_tokenize(text.lower())  # Chuyển đổi văn bản thành chữ thường và tách từ
    tokens = [token for token in tokens if token.isalpha()]  # Loại bỏ các ký tự không phải chữ cái
    tokens = [token for token in tokens if token not in stopwords.words('english')]  # Loại bỏ các từ dừng
    tokens = [token for token in tokens if token in filtered_words]
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
datapath = "./data.csv"
with open(datapath, "r") as csv_file:
  reader = csv.reader(csv_file, quotechar='"')
  documents, labels = [], []
  for idx, line in enumerate(reader):
    if idx > 0:
      documents.append(line[0])
      labels.append(line[1])

In [6]:
def filter_words(documents, min_count):
    # Tạo từ điển stop words
    stop_words = set(stopwords.words('english'))
    # Tính toán số lần xuất hiện của từng từ trong documents
    word_counts = Counter()
    for doc in documents:
        words = word_tokenize(doc.lower())
        # Loại bỏ stop words và các ký tự không phải chữ cái
        words = [word for word in words if word.isalpha() and word not in stop_words]
        word_counts.update(words)
    # Loại bỏ các từ xuất hiện dưới ngưỡng và stop words
    filtered_words = {word: count for word, count in word_counts.items() if count >= min_count}
    return filtered_words

# Sử dụng hàm filter_words để lọc ra từ điển các từ còn lại
filtered_words = filter_words(documents, 5)


In [ ]:
for idx, word in enumerate(filtered_words):
  print(idx, word)

In [8]:
docs = []
for doc in documents:
  a = tokenize_text(doc)
  b = " ".join(a)
  docs.append(b)

In [25]:
print(math.log(10))

2.302585092994046


In [58]:
def tfidf(word, document, docs):
  num_word = document.count(word)
  tf = num_word / len(tokenize_text(document))

  num_doc = 0
  for doc in docs:
    if word in doc:
      num_doc += 1
  idf = (math.log( (len(docs) + 1) / (num_doc + 1))) + 1

  return tf * idf

In [31]:
print(num_words, num_documents)

36 20


In [34]:
tfidf(indices[34], indices[36], docs)

1.6756876285817388

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tập hợp các văn bản đầu vào

# Khởi tạo TfidfVectorizer
vectorizer = TfidfVectorizer()

# Tính toán ma trận TF-IDF cho toàn bộ tập văn bản
tfidf_matrix = vectorizer.fit_transform(docs)

# Lấy danh sách các từ vựng (unique words)
unique_words = vectorizer.get_feature_names_out()

# Xác định chỉ số của từ "reviewers" trong danh sách các từ vựng
word_index = unique_words.tolist().index("one")
print(word_index)
# Lấy chỉ số TF-IDF của từ "reviewers" trong văn bản đầu tiên
tfidf_score = tfidf_matrix[0, word_index]

print("TF-IDF score của từ 'reviewers' trong văn bản đầu tiên:", tfidf_score)


638
TF-IDF score của từ 'reviewers' trong văn bản đầu tiên: 0.03131539805774427


In [10]:
print(docs[0])

one watching oz right br br first oz scenes right go show show br br oz many never far br br would say show would pretty oz first say oz well watching oz


In [11]:
num_words = len(filtered_words)
num_documents = len(docs)

indices = {idx: word for idx, word in enumerate(filtered_words)}
for idx, doc in enumerate(docs):
  indices[idx + num_words] = doc

In [ ]:
print(indices[638])

atmosphere


In [12]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Khởi tạo ma trận one-hot encoder
encoder = OneHotEncoder(sparse=False)

# Chuẩn bị dữ liệu đầu vào cho one-hot encoding
data = np.array(list(indices.values())).reshape(-1, 1)

# Thực hiện one-hot encoding
X = encoder.fit_transform(data)

# In kích thước của ma trận one-hot encoded
print("Kích thước của ma trận one-hot encoded:", X.shape)


Kích thước của ma trận one-hot encoded: (56, 56)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
word_index = unique_words.tolist().index("one")
print(word_index)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Khai báo kích thước cố định của sliding window
fixed_size = 5


# Khởi tạo từ điển sliding_window
sliding_windows = {}

# Lặp qua mỗi từ trong documents
for doc in documents:
    words = doc.split()
    # Lặp qua mỗi từ trong văn bản
    for i in range(len(words) - fixed_size):
      window = words[i:i+fixed_size]
      for word in window:
        if word in filtered_words:
          if word not in sliding_windows:
            sliding_windows[word] = 0
          sliding_windows[word] += 1



In [ ]:
# In kết quả
for word, count in sliding_windows.items():
    print("Word:", word)
    print("Number of sliding windows containing this word:", count)

In [14]:
# Khởi tạo mảng W
W = {}

# Khai báo kích thước cố định của sliding window
fixed_size = 5

# Lặp qua mỗi văn bản trong documents
for doc in documents:
    words = tokenize_text(doc)
    # Tạo set từ vựng cho mỗi văn bản để kiểm tra sự xuất hiện của từng từ
    # Lặp qua từng từ trong filtered_words
    for i in range(len(words) - fixed_size):
      windows = words[i:i+fixed_size]
      for j in range(len(windows)):
        for k in range(j):
          pair = (words[j], words[k]) if words[j] > words[k] else (words[k], words[j])
          if pair not in W:
            W[pair] = 0
          W[pair] += 1



In [ ]:
# In kết quả
for key, value in W.items():
    print("Words:", key)
    print("Number of windows containing both words:", value)

In [ ]:
print(indices[1200])

originally fan first rather disappointed watching movie view changed movie pretty funny beginning end found even though really stupid storyline portray movie br br much entertaining enjoyable movies seen saw dull way really enjoyed br br enjoyed enjoy band earth


In [ ]:
A = [[0 for i in range(num_words + num_documents)] for j in range(num_words + num_documents)]
window_size = sum([len(doc.split()) - 2 for doc in docs])

for i in range(num_words + num_documents):
  for j in range(i, num_words + num_documents):
    if i == j:
      A[i][j] = 1
    elif(i < num_words) and j < num_words:
      pair = (indices[i], indices[j]) if indices[i] < indices[j] else (indices[j], indices[i])
      A[i][j] = window_size * W[pair] / \
                  sliding_windows[indices[i]] / sliding_windows[indices[j]] if pair in W else 0
    elif i < num_words and j >= num_words:
      # if indices[i] in unique_words.tolist():
      #   word_index = unique_words.tolist().index((indices[i]))
        # print(word_index, indices[i])
        # Lấy chỉ số TF-IDF của từ "reviewers" trong văn bản đầu tiên
        # print(j-num_words, word_index)
        tfidf_score = tfidf(indices[i], indices[j], docs)
        print(tfidf_score)
        if tfidf_score > 0:
          A[i][j] = tfidf_score
    else:
      A[i][j] = 0
    A[j][i] = A[i][j]





In [60]:
arr = np.array(A)
zero_ratio = np.mean(arr == 0)
print(zero_ratio)

0.8596938775510204


In [61]:
D = [[0 for i in range(num_words + num_documents)] for j in range(num_words + num_documents)]
for i in range(num_words + num_documents):
  D[i][i] = sum([A[i][j] for j in range(num_words + num_documents)])

In [63]:
A = torch.tensor(A)
D = torch.tensor(D)
X = torch.tensor(X)

<ipython-input-63-7730f115e656>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)


In [64]:
print(D)

tensor([[2.9156, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.6303, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.6093,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 2.7415, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.7858, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 3.2250]])


In [65]:
class TextGCN(nn.Module):
  def __init__(self, A, D, vocab_size, num_outputs, num_labels):
    super().__init__()
    self.A = A
    self.D = D
    self.vocab_size = vocab_size
    self.num_outputs = num_outputs
    self.A_ = torch.matmul(torch.matmul(torch.inverse(torch.sqrt(D)), A), torch.inverse(torch.sqrt(D)))
    self.layer1 = nn.Linear(vocab_size, num_outputs)
    self.layer2 = nn.Linear(num_outputs, num_labels)
  def forward(self, X):
    layer1 = self.layer1(torch.matmul(self.A_.float(), X.float()))
    out = F.relu(layer1)
    layer2 = self.layer2(torch.matmul(self.A_, out))
    output = F.softmax(layer2)
    return output

In [66]:
vocab_size = num_words + num_documents
gcn = TextGCN(A, D, vocab_size, 10, 2)
output = gcn(X)

<ipython-input-65-7e1441e45da3>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(layer2)


In [67]:
print(output)

tensor([[0.5879, 0.4121],
        [0.5891, 0.4109],
        [0.5980, 0.4020],
        [0.5954, 0.4046],
        [0.5890, 0.4110],
        [0.5901, 0.4099],
        [0.5909, 0.4091],
        [0.5941, 0.4059],
        [0.5966, 0.4034],
        [0.5936, 0.4064],
        [0.5919, 0.4081],
        [0.5929, 0.4071],
        [0.5923, 0.4077],
        [0.5862, 0.4138],
        [0.5964, 0.4036],
        [0.5865, 0.4135],
        [0.5899, 0.4101],
        [0.5894, 0.4106],
        [0.5932, 0.4068],
        [0.5926, 0.4074],
        [0.5881, 0.4119],
        [0.5917, 0.4083],
        [0.5870, 0.4130],
        [0.5901, 0.4099],
        [0.5890, 0.4110],
        [0.5897, 0.4103],
        [0.5833, 0.4167],
        [0.5914, 0.4086],
        [0.5948, 0.4052],
        [0.5916, 0.4084],
        [0.5902, 0.4098],
        [0.5945, 0.4055],
        [0.5922, 0.4078],
        [0.5860, 0.4140],
        [0.5984, 0.4016],
        [0.5913, 0.4087],
        [0.5960, 0.4040],
        [0.5863, 0.4137],
        [0.5